## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Bing Grounding Example

This notebook demonstrates how to create an Azure AI agent that uses Bing Grounding search to find real-time information from the web.

## Features Covered:
- Setting up Bing Grounding search integration
- Using HostedWebSearchTool for web searches
- Creating agents that can access current web information
- Proper resource management with async context managers

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

This example demonstrates Azure AI agents with Bing Grounding for web search capabilities. The agent can search the web for current information and provide informed responses.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import os
from pathlib import Path
from agent_framework import ChatAgent, HostedWebSearchTool
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [2]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
# from azure.identity import InteractiveBrowserCredential  # For Azure authentication

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

# # Get tenant ID for authentication
# tenant_id = os.environ.get("TENANT_ID")
# conn_string = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

# print(f"🔑 Using Tenant ID: {tenant_id}")

# # Initialize the AI Project Client using simplified browser-based authentication
# try:
#     print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
#     # Use only InteractiveBrowserCredential with the specific tenant
#     credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
#     # Create the project client using endpoint (conn_string is actually the endpoint URL)
#     project_client = AIProjectClient(
#         endpoint=conn_string,
#         credential=credential
#     )
#     print("✅ Successfully initialized AIProjectClient")
# except Exception as e:
#     # Print error message if client initialization fails
#     print(f"❌ Error initializing project client: {str(e)}")
#     print("💡 Please complete the browser authentication prompt that should appear")

False

## Check Environment Setup

Let's verify that the required environment variable is set:



In [3]:
import os

# Check if BING_CONNECTION_ID is set
bing_connection_id = os.getenv('BING_CONNECTION_ID')
if bing_connection_id:
    print("✅ BING_CONNECTION_ID is set")
    print(f"Connection ID: {bing_connection_id[:50]}...")  # Show first 50 chars for security
else:
    print("❌ BING_CONNECTION_ID environment variable is not set")
    print("Please set this variable before proceeding.")

✅ BING_CONNECTION_ID is set
Connection ID: /subscriptions/93c5449c-bbb2-4249-9461-ddf749e0343...


## Create Bing Grounding Search Tool

Now create the `HostedWebSearchTool` with the retrieved connection ID:

In [4]:
# Create Bing Grounding search tool with the connection ID
bing_search_tool = HostedWebSearchTool(
    name="Bing Grounding Search",
    description="Search the web for current information using Bing",
    connection_id=bing_connection_id,  # Pass the retrieved connection ID
)

print("✅ Bing search tool created successfully")

✅ Bing search tool created successfully


## Create and Run the Agent

Now let's create an Azure AI agent that can use the Bing search tool to find current information from the web:

In [5]:
async def main() -> None:
    """Main function demonstrating Azure AI agent with Bing Grounding search."""
    
    # Use AzureAIAgentClient as async context manager for automatic cleanup
    async with (
        AzureAIAgentClient(async_credential=AzureCliCredential()) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        # Demonstrate agent capabilities with web search
        print("=== Azure AI Agent with Bing Grounding Search ===\n")

        user_input = "What is the most popular programming language?"
        print(f"User: {user_input}")
        response = await agent.run(user_input)
        print(f"Agent: {response.text}\n")

## Execute the Example

Run the main function to see the agent in action:

In [6]:
# Run the main function
await main()

=== Azure AI Agent with Bing Grounding Search ===

User: What is the most popular programming language?
Agent: As of February 2026, Python remains the most popular programming language, according to the TIOBE Index, with a market share of 21.81%. It leads the rankings by a significant margin over C (11.05%) and C++ (8.55%) in second and third places, respectively【3:0†source】【3:2†source】.



## Try Additional Queries

Let's test the agent with more queries that require current information:

In [7]:
async def test_more_queries():
    """Test the agent with additional queries that require web search."""
    
    async with (
        AzureAIAgentClient(async_credential=AzureCliCredential()) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        queries = [
            "What are the latest trends in artificial intelligence?",
            "What is the current weather in Seattle?",
            "What are the latest Microsoft announcements?"
        ]
        
        for query in queries:
            print(f"\n--- Query: {query} ---")
            response = await agent.run(query)
            print(f"Agent: {response.text}")
            print("-" * 50)

# Run additional queries
await test_more_queries()


--- Query: What are the latest trends in artificial intelligence? ---
Agent: The field of AI continues to evolve quickly, with key trends in 2026 including:

1. **Agentic AI**: AI systems that can independently define goals, make decisions, and complete tasks without constant human supervision are gaining traction. These agents are increasingly being integrated across industries such as supply chain management, finance, and customer support【3:4†source】.

2. **Collaboration Over Replacement**: AI is shifting from being a tool for answering questions to acting as a collaborative partner. AI assists users in tasks like content generation and data analysis, emphasizing human-AI teamwork rather than replacing human roles【3:0†source】.

3. **Generative AI at Organizational Scales**: Generative AI technologies are being employed not just individually but to transform business operations, enhancing personalization and operational efficiency【3:1†source】.

4. **Ethics and Governance**: With AI's

## Key Takeaways

1. **Bing Grounding Integration**: The `HostedWebSearchTool` provides seamless integration with Bing search capabilities
2. **Connection Retrieval**: Use the Azure AI Project Client to retrieve the full connection ID from the connection name
3. **Environment Configuration**: Set `BING_CONNECTION_NAME` in your `.env` file (just the name, not the full resource ID)
4. **Real-time Information**: Agents can now access current web information, making them more useful for time-sensitive queries
5. **Source Citation**: The agent is instructed to cite sources, promoting transparency and reliability
6. **Resource Management**: Using async context managers ensures proper cleanup of resources
7. **Tool Integration**: The Bing search tool seamlessly integrates with the agent's existing capabilities

## Troubleshooting

If you encounter issues:
1. **Authentication**: Ensure you're logged in with `az login`
2. **Connection Name**: Set `BING_CONNECTION_NAME` in your `.env` file with just the connection name (e.g., `foundrysearch`)
3. **Azure AI Project**: Verify that Bing Grounding connection is properly configured in your Azure AI Foundry project
4. **Permissions**: Ensure you have the necessary permissions to access the connected Bing search resource
5. **Connection Format**: The connection ID is automatically retrieved - you don't need to manually construct the full Azure Resource Manager path